# Step 1: Load and Merge Dataset

In [1]:
import pandas as pd

# Data files
# Replace with correct paths in your environment
files = [
    'UNSW-NB15_1.csv', 'UNSW-NB15_2.csv',
    'UNSW-NB15_3.csv', 'UNSW-NB15_4.csv'
]
features = pd.read_csv('NUSW-NB15_features.csv', encoding='ISO-8859-1')
columns = features.iloc[:, 1].tolist()

# Load and merge

In [2]:
merged_df = pd.concat([pd.read_csv(f, header=None) for f in files], ignore_index=True)
merged_df.columns = columns

C:\Users\HP\AppData\Local\Temp\ipykernel_90056\1530396623.py:1: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.concat([pd.read_csv(f, header=None) for f in files], ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_90056\1530396623.py:1: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.concat([pd.read_csv(f, header=None) for f in files], ignore_index=True)


# Step 2: Filter to 9 attack categories

In [3]:
columns_to_drop = ['id', 'srcip', 'sport', 'dstip', 'dsport', 'Label']
df_cleaned = merged_df.drop(columns=columns_to_drop, errors='ignore')

selected_attacks = [
    'Analysis', 'Backdoor', 'DoS', 'Exploits', 'Fuzzers',
    'Generic', 'Reconnaissance', 'Shellcode', 'Worms'
]
df_cleaned = df_cleaned[df_cleaned['attack_cat'].isin(selected_attacks)]

# Step 3: Clean missing/duplicates

In [4]:
df_cleaned.drop_duplicates(inplace=True)
df_cleaned.fillna(method='ffill', inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_90056\3349760250.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_cleaned.fillna(method='ffill', inplace=True)


In [6]:
# NIDS - Multi-class Classification on UNSW-NB15

# Step 1: Load and Merge Dataset
import pandas as pd

# Data files
# Replace with correct paths in your environment
files = [
    'UNSW-NB15_1.csv', 'UNSW-NB15_2.csv',
    'UNSW-NB15_3.csv', 'UNSW-NB15_4.csv'
]
features = pd.read_csv('NUSW-NB15_features.csv', encoding='ISO-8859-1')
columns = features.iloc[:, 1].tolist()

# Load and merge
merged_df = pd.concat([pd.read_csv(f, header=None) for f in files], ignore_index=True)
merged_df.columns = columns

# Step 2: Filter to 9 attack categories
columns_to_drop = ['id', 'srcip', 'sport', 'dstip', 'dsport', 'Label']
df_cleaned = merged_df.drop(columns=columns_to_drop, errors='ignore')

selected_attacks = [
    'Analysis', 'Backdoor', 'DoS', 'Exploits', 'Fuzzers',
    'Generic', 'Reconnaissance', 'Shellcode', 'Worms'
]
df_cleaned = df_cleaned[df_cleaned['attack_cat'].isin(selected_attacks)]

# Step 3: Clean missing/duplicates
df_cleaned.drop_duplicates(inplace=True)
df_cleaned.fillna(method='ffill', inplace=True)

# Step 4: Encode categorical and scale
from sklearn.preprocessing import LabelEncoder, StandardScaler

label_encoder = LabelEncoder()
df_cleaned['attack_cat'] = label_encoder.fit_transform(df_cleaned['attack_cat'])

X = df_cleaned.drop(columns=['attack_cat'])
y = df_cleaned['attack_cat']

X = pd.get_dummies(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 5: Feature selection
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(score_func=f_classif, k=40)
X_selected = selector.fit_transform(X_scaled, y)

# Step 6: Balance dataset
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_selected, y)

# Step 7: Train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled
)

# Step 8: Train models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=500),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
}

trained_models = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    trained_models[name] = model

# Step 9: Evaluate
from sklearn.metrics import classification_report, accuracy_score

for name, model in trained_models.items():
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"\n{name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Step 10: Save model
import joblib
joblib.dump(trained_models['XGBoost'], 'best_nids_model.pkl')
joblib.dump(label_encoder, 'new_label_encoder.pkl')


C:\Users\HP\AppData\Local\Temp\ipykernel_90056\1251715571.py:16: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.concat([pd.read_csv(f, header=None) for f in files], ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_90056\1251715571.py:16: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.concat([pd.read_csv(f, header=None) for f in files], ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_90056\1251715571.py:31: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_cleaned.fillna(method='ffill', inplace=True)
d:\ONGC_ML\.venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
d:\ONGC_ML\.venv


Logistic Regression Accuracy: 0.6639
                precision    recall  f1-score   support

      Analysis       0.52      0.47      0.49      8280
      Backdoor       0.45      0.74      0.56      8279
           DoS       0.48      0.08      0.14      8280
      Exploits       0.57      0.62      0.59      8280
       Generic       0.97      0.79      0.87      8280
Reconnaissance       0.76      0.91      0.83      8279
     Shellcode       0.86      0.75      0.80      8280
         Worms       0.72      0.95      0.82      8280

      accuracy                           0.66     66238
     macro avg       0.67      0.66      0.64     66238
  weighted avg       0.67      0.66      0.64     66238


Random Forest Accuracy: 0.8189
                precision    recall  f1-score   support

      Analysis       0.57      0.59      0.58      8280
      Backdoor       0.60      0.65      0.62      8279
           DoS       0.68      0.61      0.64      8280
      Exploits       0.83     

['new_label_encoder.pkl']

Step 1: Load and Merge UNSW-NB15 Dataset
### Step 1: Load and Merge Dataset

We are loading the 4 parts of the UNSW-NB15 dataset and assigning appropriate column names using the feature file. This gives us a complete dataset of ~2.5 million rows.


In [2]:
import pandas as pd

# Load feature names
features = pd.read_csv('NUSW-NB15_features.csv', encoding='ISO-8859-1')
columns = features.iloc[:, 1].tolist()

# List of dataset files
files = ['UNSW-NB15_1.csv', 'UNSW-NB15_2.csv', 'UNSW-NB15_3.csv', 'UNSW-NB15_4.csv']

# Load and merge all parts
merged_df = pd.concat([pd.read_csv(f, header=None) for f in files], ignore_index=True)
merged_df.columns = columns

# Dataset summary
print("✅ Merged dataset shape:", merged_df.shape)
print("📌 First 5 rows:")
print(merged_df.head())


C:\Users\HP\AppData\Local\Temp\ipykernel_25752\1658876593.py:11: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.concat([pd.read_csv(f, header=None) for f in files], ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_25752\1658876593.py:11: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.concat([pd.read_csv(f, header=None) for f in files], ignore_index=True)


✅ Merged dataset shape: (2540047, 49)
📌 First 5 rows:
        srcip  sport          dstip dsport proto state       dur  sbytes  \
0  59.166.0.0   1390  149.171.126.6     53   udp   CON  0.001055     132   
1  59.166.0.0  33661  149.171.126.9   1024   udp   CON  0.036133     528   
2  59.166.0.6   1464  149.171.126.7     53   udp   CON  0.001119     146   
3  59.166.0.5   3593  149.171.126.5     53   udp   CON  0.001209     132   
4  59.166.0.3  49664  149.171.126.0     53   udp   CON  0.001169     146   

   dbytes  sttl  ...  ct_ftp_cmd  ct_srv_src  ct_srv_dst ct_dst_ltm  \
0     164    31  ...           0           3           7          1   
1     304    31  ...           0           2           4          2   
2     178    31  ...           0          12           8          1   
3     164    31  ...           0           6           9          1   
4     178    31  ...           0           7           9          1   

   ct_src_ ltm  ct_src_dport_ltm  ct_dst_sport_ltm  ct_dst_src

### Step 2: Filter to 9 Attack Categories

We are keeping only the attack categories that we are interested in and dropping irrelevant network-identifying columns.


In [3]:
# Drop unwanted columns
columns_to_drop = ['id', 'srcip', 'sport', 'dstip', 'dsport', 'Label']
df_cleaned = merged_df.drop(columns=columns_to_drop, errors='ignore')

# Keep only 9 relevant attack types
selected_attacks = [
    'Analysis', 'Backdoor', 'DoS', 'Exploits', 'Fuzzers',
    'Generic', 'Reconnaissance', 'Shellcode', 'Worms'
]
df_cleaned = df_cleaned[df_cleaned['attack_cat'].isin(selected_attacks)]

print("✅ Dataset shape after filtering attack categories:", df_cleaned.shape)
print("📊 Attack category counts:")
print(df_cleaned['attack_cat'].value_counts())


✅ Dataset shape after filtering attack categories: (282987, 44)
📊 Attack category counts:
attack_cat
Generic           215481
Exploits           44525
DoS                16353
Analysis            2677
Backdoor            1795
Reconnaissance      1759
Shellcode            223
Worms                174
Name: count, dtype: int64


### Step 3: Handle Duplicates and Missing Values

We remove duplicate rows and fill any missing values using forward fill (`ffill`).


In [ ]:
# Remove duplicates
df_cleaned.drop_duplicates(inplace=True)

# Fill missing values
df_cleaned.fillna(method='ffill', inplace=True)

print("✅ Shape after cleaning missing/duplicate rows:", df_cleaned.shape)


✅ Shape after cleaning missing/duplicate rows: (64551, 44)


C:\Users\HP\AppData\Local\Temp\ipykernel_25752\375475971.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_cleaned.fillna(method='ffill', inplace=True)


### Step 4: Encode Target and Scale Features

We:
- Encode `attack_cat` using `LabelEncoder`
- Use one-hot encoding for any remaining categorical features
- Scale all numerical features using `StandardScaler`


In [5]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Encode target
label_encoder = LabelEncoder()
df_cleaned['attack_cat'] = label_encoder.fit_transform(df_cleaned['attack_cat'])

# Split features and labels
X = df_cleaned.drop(columns=['attack_cat'])
y = df_cleaned['attack_cat']

# One-hot encode categorical variables
X = pd.get_dummies(X)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("✅ Feature matrix shape (after encoding and scaling):", X_scaled.shape)


✅ Feature matrix shape (after encoding and scaling): (64551, 194)


### Step 5: Feature Selection

We select the top 40 most relevant features using ANOVA F-test with `SelectKBest`.


In [6]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(score_func=f_classif, k=40)
X_selected = selector.fit_transform(X_scaled, y)

print("✅ Shape after selecting top 40 features:", X_selected.shape)


✅ Shape after selecting top 40 features: (64551, 40)


d:\ONGC_ML\.venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
d:\ONGC_ML\.venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [ ]:
# Select a sample from your preprocessed dataset
import pandas as pd
import joblib

# Load what your model used
X_selected = pd.DataFrame(X_selected)  # assuming it's already your 40-feature matrix

# Save one row
X_selected.iloc[0].to_csv("sample_input.csv", index=False, header=False)


In [8]:
# Wrong way (vertical - 40x1):
X_selected.iloc[0].to_csv("sample_input.csv", index=False, header=False)

# ✅ Correct way (horizontal - 1x40):
X_selected.iloc[[0]].to_csv("sample_input.csv", index=False, header=False)


### Step 6: Balance Dataset Using SMOTE

We balance the number of samples across all attack categories using SMOTE (Synthetic Minority Over-sampling Technique).


In [13]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_selected, y)

print("✅ Resampled shape:", X_resampled.shape)
print("📊 Balanced class distribution:")
pd.Series(y_resampled).value_counts()


✅ Resampled shape: (220792, 40)
📊 Balanced class distribution:


attack_cat
3    27599
5    27599
2    27599
4    27599
6    27599
7    27599
0    27599
1    27599
Name: count, dtype: int64

### Step 7: Split Data into Train and Test Sets

We split the data into 70% training and 30% testing with stratification to preserve class balance.


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled
)

print(f"✅ Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")


✅ Training set shape: (154554, 40), Testing set shape: (66238, 40)


### Step 8: Train Multiple Models

We train the following classifiers:
- Logistic Regression
- Random Forest
- K-Nearest Neighbors
- XGBoost


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=500),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
}

trained_models = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    trained_models[name] = model
    print(f"✅ Trained: {name}")


d:\ONGC_ML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


✅ Trained: Logistic Regression
✅ Trained: Random Forest
✅ Trained: KNN


d:\ONGC_ML\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:46:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Trained: XGBoost


### Step 9: Evaluate Trained Models

We evaluate each model using accuracy and the classification report.


In [16]:
from sklearn.metrics import classification_report, accuracy_score

for name, model in trained_models.items():
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"\n📈 {name} Accuracy: {acc:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))



📈 Logistic Regression Accuracy: 0.6639
Classification Report:
                precision    recall  f1-score   support

      Analysis       0.52      0.47      0.49      8280
      Backdoor       0.45      0.74      0.56      8279
           DoS       0.48      0.08      0.14      8280
      Exploits       0.57      0.62      0.59      8280
       Generic       0.97      0.79      0.87      8280
Reconnaissance       0.76      0.91      0.83      8279
     Shellcode       0.86      0.75      0.80      8280
         Worms       0.72      0.95      0.82      8280

      accuracy                           0.66     66238
     macro avg       0.67      0.66      0.64     66238
  weighted avg       0.67      0.66      0.64     66238


📈 Random Forest Accuracy: 0.8193
Classification Report:
                precision    recall  f1-score   support

      Analysis       0.57      0.59      0.58      8280
      Backdoor       0.60      0.66      0.63      8279
           DoS       0.68      0.61 

### Step 10: Save the Best Model

We save the XGBoost model and label encoder for deployment.


In [17]:
import joblib

joblib.dump(trained_models['XGBoost'], 'best_nids_model.pkl')
joblib.dump(label_encoder, 'new_label_encoder.pkl')
print("✅ Model and label encoder saved successfully.")


✅ Model and label encoder saved successfully.
